In [48]:
import pandas as pd
from fredapi import Fred
import numpy as np
import yfinance as yf

In [49]:
fred=Fred(api_key="709596a84b6fac52bb342d0f8ec0c168")

In [50]:
indicators = {
    'gdp': 'GDPC1',                   # Real GDP
    'interest_rate': 'FEDFUNDS',     # Fed Funds Rate
    'unemployment': 'UNRATE',        # Unemployment Rate
    'cpi': 'CPIAUCSL',                # Consumer Price Index for All Urban Consumers
    'pce': 'PCEPI',                   # Personal Consumption Expenditures Price Index
    'nonfarm_payrolls': 'PAYEMS',    # Nonfarm Payrolls
    'corporate_profits': 'CP',       # Corporate Profits After Tax
    'money_supply': 'M2SL',          # M2 Money Stock
    'consumer_confidence': 'UMCSENT',# University of Michigan: Consumer Sentiment
    'housing_starts': 'HOUST',       # Housing Starts
    'permits': 'PERMIT'              # New Building Permits
}

In [51]:
macro_data=pd.DataFrame({key: fred.get_series(value, observation_start='2010-01-01') for key, value in indicators.items()})

In [52]:
macro_data=macro_data.shift(1)

In [53]:
key_ind=['interest_rate', 'unemployment', 'cpi', 'pce']

In [54]:
for ind in key_ind:
    macro_data[f"{ind}_lag1"]=macro_data[ind].shift(1)

rolling_features = ['interest_rate', 'cpi']
for col in rolling_features:
    macro_data[f'{col}_roll3_mean'] = macro_data[col].rolling(window=3).mean()
    macro_data[f'{col}_roll3_std'] = macro_data[col].rolling(window=3).std()

In [55]:
q_columns=['gdp','corporate_profits']

macro_data[q_columns] = np.round(macro_data[q_columns].interpolate(method='linear'),0)

In [ ]:
sp500 = yf.download("^GSPC", start="2010-01-01", end="2025-01-01")

# Obliczamy dzienne zwroty na bazie 'Close'
sp500['Return'] = sp500['Close'].pct_change()


sp500['LogReturn'] = np.log(sp500['Close'].shift(-1) / sp500['Close'])
#sp500.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [57]:
sp500.columns = ['_'.join(filter(None, col)).strip() for col in sp500.columns.values]

In [58]:
sp500.columns

Index(['Close_^GSPC', 'High_^GSPC', 'Low_^GSPC', 'Open_^GSPC', 'Volume_^GSPC',
       'Return', 'LogReturn'],
      dtype='object')

In [42]:

sp500.drop(['Close_^GSPC','High_^GSPC','Low_^GSPC','Open_^GSPC','Volume_^GSPC','Return'],axis=1,inplace=True)

In [43]:
sp500['month'] = sp500.index.to_period('M')
sp500['date'] = sp500.index
macro_data['month'] = macro_data.index.to_period('M')


In [44]:
sp500 = sp500.merge(macro_data, on='month', how='left')


In [45]:
sp500['interest_x_cpi'] = sp500['interest_rate'] * sp500['cpi']
sp500['unemp_x_gdp'] = sp500['unemployment'] * sp500['gdp']

In [46]:
sp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3768 entries, 0 to 3767
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   LogReturn                 3768 non-null   float64       
 1   month                     3768 non-null   period[M]     
 2   date                      3768 non-null   datetime64[ns]
 3   gdp                       3750 non-null   float64       
 4   interest_rate             3750 non-null   float64       
 5   unemployment              3750 non-null   float64       
 6   cpi                       3750 non-null   float64       
 7   pce                       3750 non-null   float64       
 8   nonfarm_payrolls          3750 non-null   float64       
 9   corporate_profits         3750 non-null   float64       
 10  money_supply              3750 non-null   float64       
 11  consumer_confidence       3750 non-null   float64       
 12  housing_starts      

In [47]:
sp500.to_csv("data_preped.csv",index=False)